credit : [@nagarindkx](https://github.com/nagarindkx) ดู[โค้ดเดิม](https://github.com/nagarindkx/python/blob/master/python%2006%20Sentiment%20Analysis.ipynb) และอ่าน[บทความ](https://sysadmin.psu.ac.th/2019/01/15/python-06-sentiment-analysis-with-keras-tensorflow/)




## Input

In [0]:
labels = {0:'Negative', 1:'Neutral', 2:'Positive'}
negative = [
    ['I do not like it', 0],
    ['bad movie', 0],
    ['I hate it', 0],
    ['Not good at all', 0]
]
neutral = [
    ['not bad', 1],
    ['so so', 1],
    ['OK', 1],
    ['no comment', 1]
]
positive = [
    ['Good movie', 2],
    ['I love it', 2],
    ['Like', 2],
    ['Two thumbs up', 2]
]

In [0]:
%tensorflow_version 1.x
from keras.models import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import numpy as np

Using TensorFlow backend.


In [0]:
data = np.array(negative + neutral + positive)
data

array([['bad movie', '0'],
       ['I hate it', '0'],
       ['I do not like it', '0'],
       ['Not good at all', '0'],
       ['not bad', '1'],
       ['so so', '1'],
       ['OK', '1'],
       ['no comment', '1'],
       ['Good movie', '2'],
       ['I love it', '2'],
       ['Like', '2'],
       ['Two thumbs up', '2']], dtype='<U16')

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data[:,0])
tokenizer.word_index

{'all': 12,
 'at': 11,
 'bad': 4,
 'comment': 15,
 'do': 10,
 'good': 7,
 'hate': 9,
 'i': 1,
 'it': 2,
 'like': 6,
 'love': 16,
 'movie': 5,
 'no': 14,
 'not': 3,
 'ok': 13,
 'so': 8,
 'thumbs': 18,
 'two': 17,
 'up': 19}

In [0]:
tts = tokenizer.texts_to_sequences(data[:,0])
tts

[[4, 5],
 [1, 9, 2],
 [1, 10, 3, 6, 2],
 [3, 7, 11, 12],
 [3, 4],
 [8, 8],
 [13],
 [14, 15],
 [7, 5],
 [1, 16, 2],
 [6],
 [17, 18, 19]]

In [0]:
maxlen = max([len(s) for s in tts])
x = pad_sequences(tts, maxlen=maxlen, padding="post")
x

array([[ 4,  5,  0,  0,  0],
       [ 1,  9,  2,  0,  0],
       [ 1, 10,  3,  6,  2],
       [ 3,  7, 11, 12,  0],
       [ 3,  4,  0,  0,  0],
       [ 8,  8,  0,  0,  0],
       [13,  0,  0,  0,  0],
       [14, 15,  0,  0,  0],
       [ 7,  5,  0,  0,  0],
       [ 1, 16,  2,  0,  0],
       [ 6,  0,  0,  0,  0],
       [17, 18, 19,  0,  0]], dtype=int32)

In [0]:
print("Unique lables = " , set(data[:,1]))
y = to_categorical([int(i) for i in data[:,1]])
y

Unique lables =  {'1', '2', '0'}


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

# Train

In [0]:
vocab_size = len(tokenizer.word_index) + 1

# 
np.random.seed(24)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=maxlen))
model.add(LSTM(16))
model.add(Dense(y.shape[1], activation='relu'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

model.fit(x, y, epochs=34)

y_predict = model.predict(x)
    






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 10)             200       
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                1728      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 1,979
Trainable params: 1,979
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/34





12/12 [==============================] - 1s 57ms/step - loss: 4.7406 - acc: 0.3333
Epoch 2/34
12/12 [==============================] - 0s 553us/step - loss: 3.4613 - acc: 0.3333
Epoch 3/34
12/12 [==============================] - 0s 623us/step - loss: 1.0841 - acc

# Test

In [0]:
test=[
    ['hate it',0],
    ['it is bad',0],
    ['do not like',0],
    ['just OK',1],
    ['so so',1],
    ['no idea',1],
    ['I do love it',2],
    ['Like',2],
    ['Thumbs up',2],
]
data_test = np.array(test)
x_data_test = data_test[:,0]

In [0]:
x_test=[]
for t in x_data_test:                   
    s = []                               # t :                        'I do not like it'
    for w in text_to_word_sequence(t):   # text_to_word_sequence(t) : ['I', 'do', 'not', 'like', 'it']
        s.append(tokenizer.word_index[w] if w in tokenizer.word_index else 0)
    x_test.append(s)
x_test = pad_sequences(x_test,maxlen=maxlen, padding='post')

model.predict_classes(x_test)

array([2, 0, 0, 1, 1, 1, 0, 2, 2])